In [2]:
!pip install mistralai --user
!pip install instructor  --user

     |████████████████████████████████| 440 kB 28.4 MB/s eta 0:00:01
     |████████████████████████████████| 229 kB 169.4 MB/s eta 0:00:01
     |████████████████████████████████| 160 kB 176.1 MB/s eta 0:00:01
     |████████████████████████████████| 737 kB 163.6 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
     |████████████████████████████████| 155 kB 27.0 MB/s eta 0:00:01
     |████████████████████████████████| 46 kB 7.7 MB/s s eta 0:00:01
     |████████████████████████████████| 45 kB 6.9 MB/s s eta 0:00:01
     |████████████████████████████████| 64 kB 6.4 MB/s s eta 0:00:01
     |████████████████████████████████| 243 kB 160.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 160.5 MB/s eta 0:00:01
     |████████████████████████████████| 134 kB 179.2 MB/s eta 0:00:01
     |████████████████████████████████| 946 kB 143.8 MB/s eta 0:00:01
     |████████████████████████████████| 353 kB 14

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
MISTRAL_API_KEY=os.getenv("MISTRAL_API_KEY")
mistral_api_key = os.getenv("MISTRAL_API_KEY")
fuelix_api_key =  os.getenv("FUELIX_API_KEY")

from mistralai import Mistral
from mistralai import TextChunk
import os
from instructor import from_mistral, Mode
from datetime import datetime,date
from typing import Optional
from pydantic import BaseModel, Field


In [4]:
!pip install --user elevenlabs

In [5]:
from dotenv import load_dotenv
import os

load_dotenv()
import uuid
import json
from pathlib import Path
from elevenlabs import ElevenLabs

# ------------------------
# Config
# ------------------------
ELEVENLABS_API_KEY =os.getenv("ELEVENLABS_API_KEY")
elevenlabs = ElevenLabs(api_key=ELEVENLABS_API_KEY)

# ------------------------
# Helper: ElevenLabs transcription
# ------------------------
def transcribe_with_elevenlabs(audio_path: Path):
    with open(audio_path, "rb") as f:
        transcription = elevenlabs.speech_to_text.convert(
            file=f,
            model_id="scribe_v1",
            diarize=True,
            timestamps_granularity="word",  # word-level timestamps
            tag_audio_events=False
        )

    # Convert ElevenLabs model object to dict
    if hasattr(transcription, "dict"):
        transcription = transcription.dict()
    elif hasattr(transcription, "model_dump"):
        transcription = transcription.model_dump()

    return transcription

# ------------------------
# Clean & format words
# ------------------------
def build_word_level_json(transcription):
    words = transcription.get("words", [])
    cleaned = []

    skip_tokens = {"", " "}

    for w in words:
        word_text = w.get("text", "").strip()
        if word_text in skip_tokens:
            continue

        cleaned.append({
            "word": word_text,
            "start_timestamp": w.get("start"),
            "end_timestamp": w.get("end")
        })

    return cleaned

# ------------------------
# Main
# ------------------------
def process_audio(audio_path: str, output_json: str = "transcription.json"):
    audio_path = Path(audio_path)
    if not audio_path.exists():
        raise FileNotFoundError(f"Audio file not found: {audio_path}")

    print(f"🎧 Transcribing {audio_path} ...")
    transcription = transcribe_with_elevenlabs(audio_path)
    word_json = build_word_level_json(transcription)

    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(word_json, f, indent=2, ensure_ascii=False)

    print(f"✅ Transcription saved to {output_json}")
    return word_json


# ------------------------
# Example Run
# ------------------------
if __name__ == "__main__":
    # replace with your .wav file path
    wav_file = "audio1.wav"
    process_audio(wav_file, "word_level_output.json")


🎧 Transcribing audio1.wav ...
✅ Transcription saved to word_level_output.json


In [6]:
import json
import re

# Open and read a JSON file
with open("word_level_output.json", "r") as file:
    data = json.load(file)

# Join and normalize text
clean_text = " ".join(
    re.sub(r"\s+", " ", item["word"]).strip()
    for item in data
)

print(clean_text)


How would this be for pickup or delivery? Um, pickup please. May I have your name? Tanisha. Tanisha? Yes. One moment please. Okay. May I have your good call back number please? I'm sorry, can you hear my what? I'm sorry. Uh, your phone number, your good call back number. Yes. 443-680-1162. Are you going to pick it up at the store in the 7106 Mint Railway Unit 6, in the same shopping center- Yeah. ... as Burger King, Subway, and Dunkin' Donuts? Yeah. What can I get for you today? Yes. Um, can I have a small, um, pepperoni pizza? Give me the, uh, personal pan. No, I just, I want a small size pepperoni pizza, not personal. Okay, uh, we don't have small. We only have personal pan and the medium one. Okay, I'll take the medium one then. Uh, what type of crust do you want? Is it hand sauce or thin crust? Um, pan please. Okay, one medium pan and what is your topping? Pepperoni. Okay, what else? That's it. How about our freshly baked triple chocolate brownie for dessert? No, thank you. Okay, w

In [7]:
image_ocr_markdown=clean_text #it should only contain text not timestamps (concatinate all the text in one string)

In [8]:
from instructor import from_openai, Mode
from openai import OpenAI
from datetime import datetime, date
from typing import Optional, List, Type, Any, get_origin, get_args
from pydantic import BaseModel, Field, create_model
import re

# Configuration

class FieldDefinition(BaseModel):
    name: str
    field_type: str  # "str", "Optional[str]", "datetime", "Optional[datetime]", etc.
    description: str
    alias: Optional[str] = None

class DynamicSchema(BaseModel):
    model_name: str
    context_purpose: str
    fields: List[FieldDefinition]
    reasoning: str

class DocumentExtractionUtility:
    def __init__(self, fuelix_api_key: str, schema_model: str = "gpt-4o-mini"):
        self.fuel_ix_client = from_openai(
            OpenAI(
                base_url='https://api-beta.fuelix.ai/', 
                api_key=fuelix_api_key
            ),
            mode=Mode.JSON
        )
        self.schema_model = schema_model

    def _parse_field_type(self, type_str: str) -> tuple[Type, Any]:
        """Convert string type to actual Python type and ensure it's optional"""
        type_str = type_str.strip()

        # Handle Optional types
        if type_str.startswith("Optional[") and type_str.endswith("]"):
            inner_type = type_str[9:-1]  # Remove "Optional[" and "]"
            base_type, _ = self._parse_field_type(inner_type)
            return Optional[base_type], None

        # Handle basic types
        type_mapping = {
            "str": str,
            "int": int,
            "float": float,
            "bool": bool,
            "datetime": datetime,
            "date": date,
        }

        if type_str in type_mapping:
            # Always wrap in Optional to ensure all fields are optional
            return Optional[type_mapping[type_str]], None
        else:
            # Default to Optional[str] if unknown type
            return Optional[str], None

    def create_model_from_schema(self, schema: DynamicSchema) -> Type[BaseModel]:
        """Create a Pydantic model from schema definition with all fields optional"""
        fields = {}

        for field_def in schema.fields:
            field_type, default_value = self._parse_field_type(field_def.field_type)

            # Handle aliases
            field_kwargs = {}
            if field_def.alias:
                field_kwargs['alias'] = field_def.alias

            # All fields are now optional with None as default
            fields[field_def.name] = (field_type, Field(None, description=field_def.description, **field_kwargs))

        # Create the dynamic model
        DynamicModel = create_model(schema.model_name, **fields)
        return DynamicModel

    def generate_schema(self, context: str, ocr_sample: str) -> DynamicSchema:
        """Generate extraction schema based on context and document sample"""

        schema_prompt = f"""
Context: {context}

Sample Document OCR Output:
{ocr_sample}

Based on this context and document sample, define a comprehensive data extraction schema.

Guidelines:
- Use snake_case for field names
- Available types: str, int, float, bool, datetime, date, Optional[str], Optional[int], Optional[datetime], etc.
- ALL fields should be Optional[] - this is mandatory for flexible extraction
- Use aliases for Python keywords (e.g., "from" should have alias)
- Consider what fields would be most valuable for the given context
- Include clear descriptions for each field
- Avoid making big sentences as part of the output schema. We want objective, normalized (in database terms) details like list of names, address etc
- Remember: Every single field must be Optional to handle cases where information might be missing

Generate a schema with model name, context purpose, field definitions, and reasoning.
        """

        try:
            response = self.fuel_ix_client.chat.completions.create(
                model=self.schema_model,
                response_model=DynamicSchema,
                messages=[
                    {"role": "system", "content": "You are a schema generation expert. Generate clean, practical and objective schemas for document extraction. ALL FIELDS MUST BE OPTIONAL. Examples: Optional[str] for name, Optional[int] for age, Optional[str] for phone_number, Optional[str] for pnr"},
                    {"role": "user", "content": schema_prompt}
                ],
                temperature=0,
                max_retries=5
            )
            
            # Post-process to ensure all fields are optional
            for field in response.fields:
                if not field.field_type.startswith("Optional["):
                    # Wrap non-optional types in Optional
                    field.field_type = f"Optional[{field.field_type}]"
            
            return response
        except Exception as e:
            raise Exception(f"Schema generation failed: {str(e)}")

    def extract_data(self, DynamicModel, ocr_text: str) -> BaseModel:
        """Extract data using the generated schema"""

        extraction_prompt = f"""
Extract the following information from this document:

Document OCR Text:
{ocr_text}

Extract all available fields. Use None/null for missing information.
All fields are optional, so don't worry if some information is not present in the document.
        """

        try:
            extraction_response = self.fuel_ix_client.chat.completions.create(
                model=self.schema_model,
                response_model=DynamicModel,
                messages=[
                    {"role": "system", "content": "You are a data extraction expert. Extract information accurately from documents. Set fields to None if information is not available."},
                    {"role": "user", "content": extraction_prompt}
                ],
                temperature=0,
                max_retries=5
            )
            return extraction_response
        except Exception as e:
            raise Exception(f"Data extraction failed: {str(e)}")

    def process_document(self, context: str, ocr_text: str, schema=None) -> tuple[DynamicSchema, BaseModel]:
        """Complete pipeline: generate schema and extract data"""
        if not schema:
            print(":arrows_counterclockwise: Generating extraction schema...")
            schema = self.generate_schema(context, ocr_text)

        print(f":white_check_mark: Generated schema: {schema.model_name}")
        print(f":clipboard: Fields: {len(schema.fields)} (all optional)")
        print(f":thought_balloon: Reasoning: {schema.reasoning}")
        print()

        print(":arrows_counterclockwise: Extracting data...")
        DynamicModel = self.create_model_from_schema(schema)

        extracted_data = self.extract_data(DynamicModel, ocr_text)

        print(":white_check_mark: Data extraction complete!")

        return schema, extracted_data

In [9]:
# Usage Example
# Sample OCR text (flight booking confirmation)
sample_flight_ocr =image_ocr_markdown

# Initialize the utility (you'll need to define these variables)
FUELIX_API_KEY = "ak-iN11LJkKQtOfT9681j90ICZ9dzAt"#"your-fuelix-api-key-here"
SCHEMA_MODEL = "gpt-4o-mini"  # or whatever model you want to use

extractor = DocumentExtractionUtility(FUELIX_API_KEY, SCHEMA_MODEL)

# Define context
context = "You are given noisy audio transcription output. Design schemas that can categorize and identify different types of Personally Identifiable Information (PII) which are names, phone numbers, addresses, emails only. Ensure the schema also includes a field for miscellaneous PII that may not fall into the standard categories, there can be multiple names, addresses,etc. schema should allow storing multiple names, addresses,etc for example name should be a LIST of strings like ['Anand Gurung','Anand]']"

try:
    # Process the document
    schema, extracted_data = extractor.process_document(context, sample_flight_ocr)

    print("=" * 50)
    print("GENERATED SCHEMA:")
    print("=" * 50)
    print(f"Model Name: {schema.model_name}")
    print(f"Context: {schema.context_purpose}")
    print(f"Reasoning: {schema.reasoning}")
    print("\nFields:")
    for field in schema.fields:
        alias_info = f" (alias: {field.alias})" if field.alias else ""
        print(f"  • {field.name}: {field.field_type}{alias_info}")
        print(f"    └─ {field.description}")

    print("\n" + "=" * 50)
    print("EXTRACTED DATA:")
    print("=" * 50)
    print(extracted_data.model_dump_json(indent=2))
    
except Exception as e:
    print(f":x: Error: {e}")



:arrows_counterclockwise: Generating extraction schema...
:white_check_mark: Generated schema: PIIExtractionSchema
:clipboard: Fields: 5 (all optional)
:thought_balloon: Reasoning: This schema is designed to capture various types of PII from audio transcriptions, allowing for flexible extraction of multiple entries for names, phone numbers, addresses, emails, and any other relevant information.

:arrows_counterclockwise: Extracting data...
:white_check_mark: Data extraction complete!
GENERATED SCHEMA:
Model Name: PIIExtractionSchema
Context: To extract and categorize Personally Identifiable Information (PII) from noisy audio transcription outputs.
Reasoning: This schema is designed to capture various types of PII from audio transcriptions, allowing for flexible extraction of multiple entries for names, phone numbers, addresses, emails, and any other relevant information.

Fields:
  • names: Optional[list]
    └─ List of names extracted from the transcription.
  • phone_numbers: Optiona

In [10]:
schema

DynamicSchema(model_name='PIIExtractionSchema', context_purpose='To extract and categorize Personally Identifiable Information (PII) from noisy audio transcription outputs.', fields=[FieldDefinition(name='names', field_type='Optional[list]', description='List of names extracted from the transcription.', alias=None), FieldDefinition(name='phone_numbers', field_type='Optional[list]', description='List of phone numbers extracted from the transcription.', alias=None), FieldDefinition(name='addresses', field_type='Optional[list]', description='List of addresses extracted from the transcription.', alias=None), FieldDefinition(name='emails', field_type='Optional[list]', description='List of email addresses extracted from the transcription.', alias=None), FieldDefinition(name='miscellaneous_pii', field_type='Optional[list]', description='List of miscellaneous PII that does not fall into standard categories.', alias=None)], reasoning='This schema is designed to capture various types of PII from

In [11]:
# Export schema to JSON file (2 lines)
import json
with open('schema.json', 'w') as f: json.dump(schema.model_dump(), f, indent=2)

# Import schema from JSON file (2 lines)  
with open('schema.json', 'r') as f: schema_dict = json.load(f)
schema = DynamicSchema(**schema_dict)

In [12]:
schema

DynamicSchema(model_name='PIIExtractionSchema', context_purpose='To extract and categorize Personally Identifiable Information (PII) from noisy audio transcription outputs.', fields=[FieldDefinition(name='names', field_type='Optional[list]', description='List of names extracted from the transcription.', alias=None), FieldDefinition(name='phone_numbers', field_type='Optional[list]', description='List of phone numbers extracted from the transcription.', alias=None), FieldDefinition(name='addresses', field_type='Optional[list]', description='List of addresses extracted from the transcription.', alias=None), FieldDefinition(name='emails', field_type='Optional[list]', description='List of email addresses extracted from the transcription.', alias=None), FieldDefinition(name='miscellaneous_pii', field_type='Optional[list]', description='List of miscellaneous PII that does not fall into standard categories.', alias=None)], reasoning='This schema is designed to capture various types of PII from

# Using The Schema Again

In [13]:
extracted_data

PIIExtractionSchema(names='Tanisha', phone_numbers='443-680-1162', addresses='7106 Mint Railway Unit 6', emails=None, miscellaneous_pii=None)

In [14]:
import json
import re
from pathlib import Path

# Example: path to your transcription file
transcription_path = Path("word_level_output.json")

# Load the JSON transcript
with open(transcription_path, "r") as f:
    transcript = json.load(f)

# Convert Pydantic model to dict
data_dict = extracted_data.dict()   # if Pydantic v1
# data_dict = extracted_data.model_dump()  # if Pydantic v2

# Collect all searchable words from extracted_data (split multi-word strings)
search_terms = set()
for v in data_dict.values():
    if v and isinstance(v, str):
        # Split by spaces
        words = v.split()
        # Normalize: lowercase, remove punctuation
        words = [re.sub(r"[^\w]", "", w.lower()) for w in words]
        search_terms.update(words)

# Extract timestamps of words in transcript matching any search term
timestamps = []
for entry in transcript:
    # Normalize transcript word: lowercase + remove punctuation
    normalized_word = re.sub(r"[^\w]", "", entry["word"].lower())
    if normalized_word in search_terms:
        timestamps.append({
            
            "start_timestamp": entry["start_timestamp"],
            "end_timestamp": entry["end_timestamp"]
        })

print(timestamps)


[{'start_timestamp': 10.18, 'end_timestamp': 12.2}, {'start_timestamp': 12.24, 'end_timestamp': 14.14}, {'start_timestamp': 29.48, 'end_timestamp': 34.02}, {'start_timestamp': 36.86, 'end_timestamp': 37.66}, {'start_timestamp': 37.72, 'end_timestamp': 37.94}, {'start_timestamp': 37.96, 'end_timestamp': 38.34}, {'start_timestamp': 38.42, 'end_timestamp': 38.68}, {'start_timestamp': 39.06, 'end_timestamp': 39.14}]


/tmp/ipykernel_388/3991066742.py:13: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  data_dict = extracted_data.dict()   # if Pydantic v1


In [15]:
import json
import numpy as np
import wave
import struct
from typing import List, Dict

def generate_beep(sample_rate: int, duration: float = 0.1, frequency: int = 1000) -> np.ndarray:
    """
    Generate a beep sound as a numpy array.
    """
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    beep = np.sin(2 * np.pi * frequency * t)
    fade_samples = int(sample_rate * 0.01)  # 10ms fade
    if fade_samples > 0 and len(beep) > fade_samples * 2:
        beep[:fade_samples] *= np.linspace(0, 1, fade_samples)
        beep[-fade_samples:] *= np.linspace(1, 0, fade_samples)
    return beep

def read_wav_file(filename: str) -> tuple:
    """
    Read WAV file and return audio data and parameters.
    """
    with wave.open(filename, 'rb') as wav_file:
        sample_rate = wav_file.getframerate()
        num_channels = wav_file.getnchannels()
        sample_width = wav_file.getsampwidth()
        num_frames = wav_file.getnframes()
        raw_audio = wav_file.readframes(num_frames)
        
        if sample_width == 1:
            dtype = np.uint8
            audio_data = np.frombuffer(raw_audio, dtype=dtype)
            audio_data = audio_data.astype(np.float32) / 127.5 - 1.0
        elif sample_width == 2:
            dtype = np.int16
            audio_data = np.frombuffer(raw_audio, dtype=dtype)
            audio_data = audio_data.astype(np.float32) / 32767.0
        elif sample_width == 4:
            dtype = np.int32
            audio_data = np.frombuffer(raw_audio, dtype=dtype)
            audio_data = audio_data.astype(np.float32) / 2147483647.0
        else:
            raise ValueError(f"Unsupported sample width: {sample_width}")
        
        if num_channels > 1:
            audio_data = audio_data.reshape(-1, num_channels)
    
    return audio_data, sample_rate, num_channels, sample_width

def write_wav_file(filename: str, audio_data: np.ndarray, sample_rate: int, 
                   num_channels: int, sample_width: int):
    """
    Write audio data to WAV file.
    """
    if sample_width == 1:
        audio_int = ((audio_data + 1.0) * 127.5).astype(np.uint8)
    elif sample_width == 2:
        audio_int = (audio_data * 32767.0).astype(np.int16)
    elif sample_width == 4:
        audio_int = (audio_data * 2147483647.0).astype(np.int32)
    
    with wave.open(filename, 'wb') as wav_file:
        wav_file.setnchannels(num_channels)
        wav_file.setsampwidth(sample_width)
        wav_file.setframerate(sample_rate)
        wav_file.writeframes(audio_int.tobytes())

def merge_close_timestamps(timestamps: List[Dict], min_gap: float = 0.1) -> List[Dict]:
    """
    Merge beeps if the gap between end of one and start of next < min_gap seconds.
    """
    if not timestamps:
        return []

    merged = [timestamps[0]]
    for ts in timestamps[1:]:
        last = merged[-1]
        if ts["start_timestamp"] - last["end_timestamp"] < min_gap:
            print(f"Merging beep [{last['start_timestamp']:.2f}-{last['end_timestamp']:.2f}] "
                  f"with [{ts['start_timestamp']:.2f}-{ts['end_timestamp']:.2f}]")
            last["end_timestamp"] = max(last["end_timestamp"], ts["end_timestamp"])
        else:
            merged.append(ts)
    return merged

def add_beeps_to_wav(input_file: str, output_file: str, timestamps_json: str, 
                     beep_volume: float = 0.3, beep_frequency: int = 1000):
    """
    Replace original audio with beeps at specified timestamps.
    """
    try:
        timestamps = json.loads(timestamps_json)
    except json.JSONDecodeError:
        with open(timestamps_json, 'r') as f:
            timestamps = json.load(f)

    # Merge close timestamps (<100ms apart)
    timestamps = merge_close_timestamps(timestamps, min_gap=0.1)

    print(f"Reading WAV file: {input_file}")
    audio_data, sample_rate, num_channels, sample_width = read_wav_file(input_file)

    print(f"Audio info: {len(audio_data)/sample_rate:.2f}s, "
          f"{sample_rate}Hz, {num_channels} channels")

    audio_duration = len(audio_data) / sample_rate

    for i, timestamp in enumerate(timestamps):
        start_time = timestamp['start_timestamp']
        end_time = timestamp['end_timestamp']

        # Clamp inside audio range
        start_time = max(0.0, min(start_time, audio_duration))
        end_time = max(0.0, min(end_time, audio_duration))

        if end_time <= start_time:
            print(f"Skipping beep {i+1}: invalid/zero duration")
            continue

        print(f"Adding beep {i+1}: {start_time:.2f}s - {end_time:.2f}s")

        start_sample = int(start_time * sample_rate)
        end_sample = int(end_time * sample_rate)

        beep_duration = (end_sample - start_sample) / sample_rate
        if beep_duration <= 0:
            print(f"Skipping beep {i+1}: invalid duration {beep_duration}")
            continue

        beep = generate_beep(sample_rate, beep_duration, beep_frequency) * beep_volume
        beep_length = min(len(beep), end_sample - start_sample)

        # First mute the original audio
        if num_channels == 1:
            audio_data[start_sample:start_sample + beep_length] = 0
            audio_data[start_sample:start_sample + beep_length] += beep[:beep_length]
        else:
            audio_data[start_sample:start_sample + beep_length, :] = 0
            for channel in range(num_channels):
                audio_data[start_sample:start_sample + beep_length, channel] += beep[:beep_length]

    # Normalize if clipping
    max_val = np.max(np.abs(audio_data))
    if max_val > 1.0:
        print(f"Normalizing audio (max value was {max_val:.3f})")
        audio_data = audio_data / max_val

    print(f"Writing output file: {output_file}")
    write_wav_file(output_file, audio_data, sample_rate, num_channels, sample_width)
    print("Done!")

def main():
    """
    Example usage of the beep inserter.
    """
    input_wav_file = "audio1.wav"  
    output_wav_file = "drive_thru_beep.wav"  

    try:
        add_beeps_to_wav(
            input_file=input_wav_file,
            output_file=output_wav_file,
            timestamps_json=json.dumps(timestamps),  # pass as JSON string
            beep_volume=0.3,
            beep_frequency=1000
        )
    except FileNotFoundError as e:
        print(f"Error: File not found - {e}")
        print("Please make sure your input audio file exists.")
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()


Merging beep [10.18-12.20] with [12.24-14.14]
Merging beep [36.86-37.66] with [37.72-37.94]
Merging beep [36.86-37.94] with [37.96-38.34]
Merging beep [36.86-38.34] with [38.42-38.68]
Reading WAV file: audio1.wav
Audio info: 135.40s, 8000Hz, 1 channels
Adding beep 1: 10.18s - 14.14s
Adding beep 2: 29.48s - 34.02s
Adding beep 3: 36.86s - 38.68s
Adding beep 4: 39.06s - 39.14s
Writing output file: drive_thru_beep.wav
Done!
